In [ ]:
import gc
import os
import time
import warnings
from itertools import combinations
from warnings import simplefilter

import joblib
import xgboost as xgb
import numpy as np
import pandas as pd
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import KFold, TimeSeriesSplit

warnings.filterwarnings("ignore")
simplefilter(action="ignore", category=pd.errors.PerformanceWarning)

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
import os
os.environ['KAGGLE_CONFIG_DIR'] = "/content/gdrive/My Drive/Kaggle" # https://drive.google.com/drive/folders/18KshTFZ6gGQMeqUf5N6FZq2H1P9fNYlB?usp=sharing

In [ ]:
!kaggle competitions download optiver-trading-at-the-close
!unzip optiver-trading-at-the-close.zip

 97% 194M/201M [00:02<00:00, 78.6MB/s]
100% 201M/201M [00:02<00:00, 87.0MB/s]
Archive:  optiver-trading-at-the-close.zip
  inflating: example_test_files/revealed_targets.csv  
  inflating: example_test_files/sample_submission.csv  
  inflating: example_test_files/test.csv  
  inflating: optiver2023/__init__.py  
  inflating: optiver2023/competition.cpython-310-x86_64-linux-gnu.so  
  inflating: public_timeseries_testing_util.py  
  inflating: train.csv               


In [ ]:

df = pd.read_csv("/content/train.csv")

In [ ]:
def reduce_mem_usage(df, verbose=0):
    """
    Iterate through all numeric columns of a dataframe and modify the data type
    to reduce memory usage.
    """

    start_mem = df.memory_usage().sum() / 1024**2

    for col in df.columns:
        col_type = df[col].dtype

        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == "int":
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float32)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float32)

    if verbose:
        logger.info(f"Memory usage of dataframe is {start_mem:.2f} MB")
        end_mem = df.memory_usage().sum() / 1024**2
        logger.info(f"Memory usage after optimization is: {end_mem:.2f} MB")
        decrease = 100 * (start_mem - end_mem) / start_mem
        logger.info(f"Decreased by {decrease:.2f}%")

    return df

from numba import njit, prange

@njit(parallel=True)
def compute_triplet_imbalance(df_values, comb_indices):
    num_rows = df_values.shape[0]
    num_combinations = len(comb_indices)
    imbalance_features = np.empty((num_rows, num_combinations))

    for i in prange(num_combinations):
        a, b, c = comb_indices[i]
        for j in range(num_rows):
            max_val = max(df_values[j, a], df_values[j, b], df_values[j, c])
            min_val = min(df_values[j, a], df_values[j, b], df_values[j, c])
            mid_val = df_values[j, a] + df_values[j, b] + df_values[j, c] - min_val - max_val
            if mid_val == min_val:  # Prevent division by zero
                imbalance_features[j, i] = np.nan
            else:
                imbalance_features[j, i] = (max_val - mid_val) / (mid_val - min_val)

    return imbalance_features


def calculate_triplet_imbalance_numba(price, df):
    # Convert DataFrame to numpy array for Numba compatibility
    df_values = df[price].values
    comb_indices = [(price.index(a), price.index(b), price.index(c)) for a, b, c in combinations(price, 3)]

    # Calculate the triplet imbalance
    features_array = compute_triplet_imbalance(df_values, comb_indices)

    # Create a DataFrame from the results
    columns = [f"{a}_{b}_{c}_imb2" for a, b, c in combinations(price, 3)]
    features = pd.DataFrame(features_array, columns=columns)

    return features

# generate imbalance features
def imbalance_features(df):
    prices = ["reference_price", "far_price", "near_price", "ask_price", "bid_price", "wap"]
    sizes = ["matched_size", "bid_size", "ask_size", "imbalance_size"]

    # V1
    df["volume"] = df.eval("ask_size + bid_size")
    df["mid_price"] = df.eval("(ask_price + bid_price) / 2")
    df["liquidity_imbalance"] = df.eval("(bid_size-ask_size)/(bid_size+ask_size)")
    df["matched_imbalance"] = df.eval("(imbalance_size-matched_size)/(matched_size+imbalance_size)")
    df["size_imbalance"] = df.eval("bid_size / ask_size")

    for c in combinations(prices, 2):
        df[f"{c[0]}_{c[1]}_imb"] = df.eval(f"({c[0]} - {c[1]})/({c[0]} + {c[1]})")

    for c in [['ask_price', 'bid_price', 'wap', 'reference_price'], sizes]:
        triplet_feature = calculate_triplet_imbalance_numba(c, df)
        df[triplet_feature.columns] = triplet_feature.values

    # V2
    df["stock_weights"] = df["stock_id"].map(weights)
    df["weighted_wap"] = df["stock_weights"] * df["wap"]
    df['wap_momentum'] = df.groupby('stock_id')['weighted_wap'].pct_change(periods=6)
    df["imbalance_momentum"] = df.groupby(['stock_id'])['imbalance_size'].diff(periods=1) / df['matched_size']
    df["price_spread"] = df["ask_price"] - df["bid_price"]
    df["spread_intensity"] = df.groupby(['stock_id'])['price_spread'].diff()
    df['price_pressure'] = df['imbalance_size'] * (df['ask_price'] - df['bid_price'])
    df['market_urgency'] = df['price_spread'] * df['liquidity_imbalance']
    df['depth_pressure'] = (df['ask_size'] - df['bid_size']) * (df['far_price'] - df['near_price'])
    df['spread_depth_ratio'] = (df['ask_price'] - df['bid_price']) / (df['bid_size'] + df['ask_size'])
    df['mid_price_movement'] = df['mid_price'].diff(periods=5).apply(lambda x: 1 if x > 0 else (-1 if x < 0 else 0))
    df['micro_price'] = ((df['bid_price'] * df['ask_size']) + (df['ask_price'] * df['bid_size'])) / (df['bid_size'] + df['ask_size'])
    df['relative_spread'] = (df['ask_price'] - df['bid_price']) / df['wap']

    for func in ["mean", "std", "skew", "kurt"]:
        df[f"all_prices_{func}"] = df[prices].agg(func, axis=1)
        df[f"all_sizes_{func}"] = df[sizes].agg(func, axis=1)

    # V3
    for col in ['matched_size', 'imbalance_size', 'reference_price', 'imbalance_buy_sell_flag']:
        for window in [1, 2, 3, 5, 10]:
            df[f"{col}_shift_{window}"] = df.groupby('stock_id')[col].shift(window)
            df[f"{col}_ret_{window}"] = df.groupby('stock_id')[col].pct_change(window)

    for col in ['ask_price', 'bid_price', 'ask_size', 'bid_size',
                'wap', 'near_price', 'far_price']:
        for window in [1, 2, 3, 5, 10]:
            df[f"{col}_diff_{window}"] = df.groupby("stock_id")[col].diff(window)

    return df.replace([np.inf, -np.inf], 0)

# generate time & stock features
def other_features(df):
    df["dow"] = df["date_id"] % 5
    df["dom"] = df["date_id"] % 20
    df["seconds"] = df["seconds_in_bucket"] % 60
    df["minute"] = df["seconds_in_bucket"] // 60

    for key, value in global_stock_id_feats.items():
        df[f"global_{key}"] = df["stock_id"].map(value.to_dict())

    return df

# generate all features
def generate_all_features(df):
    #cols = [c for c in df.columns if c not in ["row_id", "time_id", "target"]]
    #df = df[cols]
    df = imbalance_features(df)
    df = other_features(df)
    gc.collect()

    #feature_name = [i for i in df.columns if i not in ["row_id", "target", "time_id", "date_id"]]
    return df
    #return df[feature_name]

weights = [
    0.004, 0.001, 0.002, 0.006, 0.004, 0.004, 0.002, 0.006, 0.006, 0.002, 0.002, 0.008,
    0.006, 0.002, 0.008, 0.006, 0.002, 0.006, 0.004, 0.002, 0.004, 0.001, 0.006, 0.004,
    0.002, 0.002, 0.004, 0.002, 0.004, 0.004, 0.001, 0.001, 0.002, 0.002, 0.006, 0.004,
    0.004, 0.004, 0.006, 0.002, 0.002, 0.04, 0.002, 0.002, 0.004, 0.04 , 0.002, 0.001,
    0.006, 0.004, 0.004, 0.006, 0.001, 0.004, 0.004, 0.002, 0.006, 0.004, 0.006, 0.004,
    0.006, 0.004, 0.002, 0.001, 0.002, 0.004, 0.002, 0.008, 0.004, 0.004, 0.002, 0.004,
    0.006, 0.002, 0.004, 0.004, 0.002, 0.004, 0.004, 0.004, 0.001, 0.002, 0.002, 0.008,
    0.02 , 0.004, 0.006, 0.002, 0.02 , 0.002, 0.002, 0.006, 0.004, 0.002, 0.001, 0.02,
    0.006, 0.001, 0.002, 0.004, 0.001, 0.002, 0.006, 0.006, 0.004, 0.006, 0.001, 0.002,
    0.004, 0.006, 0.006, 0.001, 0.04 , 0.006, 0.002, 0.004, 0.002, 0.002, 0.006, 0.002,
    0.002, 0.004, 0.006, 0.006, 0.002, 0.002, 0.008, 0.006, 0.004, 0.002, 0.006, 0.002,
    0.004, 0.006, 0.002, 0.004, 0.001, 0.004, 0.002, 0.004, 0.008, 0.006, 0.008, 0.002,
    0.004, 0.002, 0.001, 0.004, 0.004, 0.004, 0.006, 0.008, 0.004, 0.001, 0.001, 0.002,
    0.006, 0.004, 0.001, 0.002, 0.006, 0.004, 0.006, 0.008, 0.002, 0.002, 0.004, 0.002,
    0.04 , 0.002, 0.002, 0.004, 0.002, 0.002, 0.006, 0.02 , 0.004, 0.002, 0.006, 0.02,
    0.001, 0.002, 0.006, 0.004, 0.006, 0.004, 0.004, 0.004, 0.004, 0.002, 0.004, 0.04,
    0.002, 0.008, 0.002, 0.004, 0.001, 0.004, 0.006, 0.004,
]

weights = {int(k):v for k,v in enumerate(weights)}

In [ ]:
global_stock_id_feats = {
        "median_size": df.groupby("stock_id")["bid_size"].median() + df.groupby("stock_id")["ask_size"].median(),
        "std_size": df.groupby("stock_id")["bid_size"].std() + df.groupby("stock_id")["ask_size"].std(),
        "ptp_size": df.groupby("stock_id")["bid_size"].max() - df.groupby("stock_id")["bid_size"].min(),
        "median_price": df.groupby("stock_id")["bid_price"].median() + df.groupby("stock_id")["ask_price"].median(),
        "std_price": df.groupby("stock_id")["bid_price"].std() + df.groupby("stock_id")["ask_price"].std(),
        "ptp_price": df.groupby("stock_id")["bid_price"].max() - df.groupby("stock_id")["ask_price"].min(),
  }

In [ ]:
# split_day = 435
# df_train = df[df["date_id"] <= split_day]
# df_valid = df[df["date_id"] > split_day]

# global_stock_id_feats = {
#         "median_size": df_train.groupby("stock_id")["bid_size"].median() + df_train.groupby("stock_id")["ask_size"].median(),
#         "std_size": df_train.groupby("stock_id")["bid_size"].std() + df_train.groupby("stock_id")["ask_size"].std(),
#         "ptp_size": df_train.groupby("stock_id")["bid_size"].max() - df_train.groupby("stock_id")["bid_size"].min(),
#         "median_price": df_train.groupby("stock_id")["bid_price"].median() + df_train.groupby("stock_id")["ask_price"].median(),
#         "std_price": df_train.groupby("stock_id")["bid_price"].std() + df_train.groupby("stock_id")["ask_price"].std(),
#         "ptp_price": df_train.groupby("stock_id")["bid_price"].max() - df_train.groupby("stock_id")["ask_price"].min(),
#   }


In [ ]:
# df_train = df
# df_train_feats = generate_all_features(df_train)
# df_train_feats = reduce_mem_usage(df_train_feats)

# offline_split = df_train['date_id']>(split_day - 45)
# df_offline_train = df_train_feats[~offline_split]
# df_offline_valid = df_train_feats[offline_split]
# df_offline_train_target = df_train['target'][~offline_split]
# df_offline_valid_target = df_train['target'][offline_split]

# # Check and remove NaN values
# df_offline_train_target = df_offline_train_target.dropna()
# df_offline_valid_target = df_offline_valid_target.dropna()

# # Check and remove infinite values, if any
# df_offline_train_target.replace([np.inf, -np.inf], np.nan, inplace=True)
# df_offline_valid_target.replace([np.inf, -np.inf], np.nan, inplace=True)

# df_offline_train_target.dropna(inplace=True)
# df_offline_valid_target.dropna(inplace=True)

# df_offline_train = df_offline_train.loc[df_offline_train_target.index]
# df_offline_valid = df_offline_valid.loc[df_offline_valid_target.index]

# # Function to remove NaN and Inf values from DataFrame
# def remove_nan_inf(df):
#     df.replace([np.inf, -np.inf], np.nan, inplace=True)  # Replace Inf with NaN
#     df.dropna(inplace=True)  # Drop all NaNs
#     return df

# # Clean the data for both features and targets
# df_offline_train = remove_nan_inf(df_offline_train)
# df_offline_valid = remove_nan_inf(df_offline_valid)
# df_offline_train_target = remove_nan_inf(df_offline_train_target)
# df_offline_valid_target = remove_nan_inf(df_offline_valid_target)

# # Ensure the indices of features and targets match
# df_offline_train_target = df_offline_train_target.loc[df_offline_train.index]
# df_offline_valid_target = df_offline_valid_target.loc[df_offline_valid.index]
# df_offline_train = df_offline_train.reindex(df_offline_train_target.index)
# df_offline_valid = df_offline_valid.reindex(df_offline_valid_target.index)

# X_train = torch.tensor(df_offline_train.values, dtype=torch.float32)
# y_train = torch.tensor(df_offline_train_target.values, dtype=torch.float32)
# X_val = torch.tensor(df_offline_valid.values, dtype=torch.float32)
# y_val = torch.tensor(df_offline_valid_target.values, dtype=torch.float32)


In [ ]:
def imputer(df):
    far_price_mean = df['far_price'].mean()
    near_price_mean = df['near_price'].mean()
    df['far_price'] = df['far_price'].fillna(far_price_mean)
    df['near_price'] = df['near_price'].fillna(near_price_mean)

    return df, far_price_mean, near_price_mean

def add_missing_data(df):
    all_stock_ids = set(range(200))
    all_missed_data_list = []

    grouped = df.groupby('time_id')

    for t, group in grouped:
        current_stock_ids = set(group['stock_id'].to_list())
        missed_stock_id = list(all_stock_ids - current_stock_ids)

        date_id = group['date_id'].iloc[-1]
        seconds_in_bucket = group['seconds_in_bucket'].iloc[-1]

        missed_stock_id_num = len(missed_stock_id)
        missed_date_id = [date_id] * missed_stock_id_num
        missed_seconds_in_bucket = [seconds_in_bucket] * missed_stock_id_num
        missed_time_id = [t] * missed_stock_id_num

        missed_data = pd.DataFrame({
            'stock_id': missed_stock_id,
            'date_id': missed_date_id,
            'seconds_in_bucket': missed_seconds_in_bucket,
            'time_id': missed_time_id
        })

        all_missed_data_list.append(missed_data)

    all_missed_data = pd.concat(all_missed_data_list, axis=0).reset_index(drop=True).astype(int)

    df = pd.concat([df, all_missed_data], axis=0)
    df = df.sort_values(by=['time_id', 'stock_id']).reset_index(drop=True)
    df = df.groupby('stock_id').apply(lambda x: x.fillna(method='bfill')).reset_index(drop=True)

    return df

train, far_price_mean, near_price_mean = imputer(df)
train = add_missing_data(df)

def sizesum_and_pricestd(df):
    price_ftrs = ['reference_price', 'far_price', 'near_price', 'bid_price', 'ask_price', 'wap'] # std
    size_ftrs = ['imbalance_size', 'matched_size', 'bid_size', 'ask_size'] # sum

    rolled = df[['stock_id'] + size_ftrs].groupby('stock_id').rolling(window=6, min_periods=1).sum()
    rolled = rolled.reset_index(level=0, drop=True)
    for col in size_ftrs:
        df[f'{col}_rolled_sum'] = rolled[col]

    rolled = df[['stock_id'] + price_ftrs].groupby('stock_id').rolling(window=6, min_periods=1).std().fillna(0)
    rolled = rolled.reset_index(level=0, drop=True)
    for col in price_ftrs:
        df[f'{col}_rolled_std'] = rolled[col]

    return df

train = sizesum_and_pricestd(train)

def remove_element(input_list, drop_list):
    return [e for e in input_list if e not in drop_list]

no_feature_cols = ['date_id', 'row_id', 'time_id', 'target', 'currently_scored']

feature_cols = remove_element(train.columns, no_feature_cols)
target_col = 'target'

In [ ]:
avg = train[feature_cols].mean()
std = train[feature_cols].std()

train[feature_cols] = (train[feature_cols] - avg)/std

In [ ]:
train = train.astype('float32')
train = reduce_mem_usage(train)

seq_len = 16

# Grouping by time_id
grouped_by_time = train.groupby('stock_id')

def generate_data(grouped_by_time, seq_len):
    for _, group in grouped_by_time:
        # Sorting by stock_id to maintain consistency across images
        group_sorted = group.sort_values(by='time_id')

        features = group_sorted[feature_cols].values

        windows = []

        for t in range(0, seq_len - 1):
            copy_0 = np.stack([features[0]] * (seq_len - 1 - t))
            cut_0 = features[: t + 1]
            windows.append(np.vstack((copy_0, cut_0)))

        for t in range(0, features.shape[0] - seq_len + 1):
            windows.append(features[t: t+seq_len, :])

        # Convert list of windows to numpy array
        features_array = np.stack(windows)

        target = group_sorted['target'].values

        # Yield the result for this group to avoid storing all results in memory
        yield features_array, target

# Use generator to iterate over data
data_generator = generate_data(grouped_by_time, seq_len=seq_len)

# If you need to store results in arrays:
datas, labels = zip(*data_generator)
data = np.array(datas).reshape(-1, seq_len, len(feature_cols))
label = np.array(labels).reshape(-1,)

In [ ]:
import torch
from torch.utils.data import DataLoader, TensorDataset, Subset, random_split

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
torch.manual_seed(42)

data = torch.tensor(data, dtype=torch.float32).to(device)
label = torch.tensor(label, dtype=torch.float32).to(device)

dataset = TensorDataset(data, label)

train_ratio = 0.8
train_size = int(train_ratio * len(dataset))
valid_size = len(dataset) - train_size
train_dataset, valid_dataset = random_split(dataset, [train_size, valid_size])

batch_size = 1024

train_loader = DataLoader(train_dataset, batch_size=batch_size)
valid_loader = DataLoader(valid_dataset, batch_size=batch_size)

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset, Subset, random_split
from torch.optim.lr_scheduler import ReduceLROnPlateau

In [ ]:
# import torch
# import torch.nn as nn
# import torch.nn.functional as F
# import torch.optim as optim
# from torch.utils.data import DataLoader, TensorDataset, Subset, random_split
# from torch.optim.lr_scheduler import ReduceLROnPlateau

# class MyModel(nn.Module):
#     def __init__(self, feature_num, d_model, nhead, num_layers):
#         super(MyModel, self).__init__()
#         self.embedding = nn.Linear(feature_num, d_model)
#         self.tf1 = nn.Transformer(d_model=d_model, nhead=nhead, num_encoder_layers=num_layers, batch_first=True)
#         self.fc = nn.Linear(d_model, d_model)
#         self.dropout = nn.Dropout(0.5)
#         self.tf2 = nn.Transformer(d_model=d_model, nhead=nhead, num_encoder_layers=num_layers, batch_first=True)
#         self.decoder = nn.Linear(d_model, 1)

#     def forward(self, x):
#         x = self.embedding(x)
#         x = self.tf1.encoder(x)
#         x = x[:, -1, :]
#         x = self.fc(x)
#         x = self.dropout(x)
#         x = self.tf2.encoder(x)
#         x = self.decoder(x)

#         return x

In [ ]:
# class TrendExtractor(nn.Module):
#     """
#     A simplified trend extractor. In a real Autoformer, this would be more complex,
#     possibly using a decomposition method.
#     """
#     def __init__(self, input_dim, hidden_dim):
#         super(TrendExtractor, self).__init__()
#         self.linear = nn.Linear(input_dim, hidden_dim)

#     def forward(self, x):
#         return self.linear(x)

# class SimplifiedAutoformerLayer(nn.Module):
#     """
#     A simplified version of an Autoformer layer. In the actual Autoformer,
#     this would include an efficient attention mechanism and other components
#     specific to time series data.
#     """
#     def __init__(self, d_model, nhead):
#         super(SimplifiedAutoformerLayer, self).__init__()
#         self.self_attn = nn.MultiheadAttention(d_model, nhead)
#         self.linear = nn.Linear(d_model, d_model)

#     def forward(self, x):
#         attn_output, _ = self.self_attn(x, x, x)
#         return F.relu(self.linear(attn_output))

# class MyModel(nn.Module):
#     def __init__(self, feature_num, d_model, nhead, num_layers):
#         super(MyModel, self).__init__()
#         self.embedding = nn.Linear(feature_num, d_model)

#         # Trend extraction layer
#         self.trend_extractor = TrendExtractor(d_model, d_model)

#         # Autoformer layers
#         self.autoformer_layers = nn.ModuleList([SimplifiedAutoformerLayer(d_model, nhead) for _ in range(num_layers)])

#         self.fc = nn.Linear(d_model, d_model)
#         self.dropout = nn.Dropout(0.5)

#         # Final decoder layer
#         self.decoder = nn.Linear(d_model, 1)

#     def forward(self, x):
#         x = self.embedding(x)

#         # Trend extraction
#         trend = self.trend_extractor(x)

#         # Autoformer layers
#         for layer in self.autoformer_layers:
#             x = layer(x)

#         x = x + trend  # Combining with the trend component
#         x = x[:, -1, :]  # Assuming we need the last timestep
#         x = self.fc(x)
#         x = self.dropout(x)
#         x = self.decoder(x)

#         return x

In [ ]:
class LongRangeTransformerLayer(nn.Module):
    def __init__(self, d_model, nhead):
        super(LongRangeTransformerLayer, self).__init__()
        self.self_attn = nn.MultiheadAttention(d_model, nhead)
        self.linear1 = nn.Linear(d_model, d_model)
        self.linear2 = nn.Linear(d_model, d_model)
        self.norm1 = nn.LayerNorm(d_model)
        self.norm2 = nn.LayerNorm(d_model)
        self.dropout = nn.Dropout(0.1)

    def forward(self, x):
        x = x + self.dropout(self.self_attn(x, x, x)[0])
        x = self.norm1(x)

        x = x + self.dropout(F.relu(self.linear1(x)))
        x = self.norm2(self.linear2(x))

        return x

class MyModel(nn.Module):
    def __init__(self, feature_num, d_model, nhead, num_layers):
        super(MyModel, self).__init__()
        self.embedding = nn.Linear(feature_num, d_model)

        self.long_range_transformer_layers = nn.ModuleList([LongRangeTransformerLayer(d_model, nhead) for _ in range(num_layers)])

        self.fc = nn.Linear(d_model, d_model)
        self.dropout = nn.Dropout(0.5)

        self.decoder = nn.Linear(d_model, 1)

    def forward(self, x):
        x = self.embedding(x)

        for layer in self.long_range_transformer_layers:
            x = layer(x)

        x = x[:, -1, :]
        x = self.fc(x)
        x = self.dropout(x)
        x = self.decoder(x)

        return x

In [ ]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_squared_log_error, r2_score
import math

tot_train_losses = []
tot_train_mae = []
tot_train_rmse = []
tot_train_rmsle = []

tot_valid_losses = []
tot_valid_mae = []
tot_valid_rmse = []
tot_valid_rmsle = []

is_train = True
if is_train:
    input_size = data.shape[-1]

    n_epochs = 50
    lr = 1e-03

    pre_epoch_valid_mae = np.inf
    patience_counter = 0

    model = MyModel(feature_num=input_size, d_model=64, nhead=2, num_layers=1).to(device)
    #model = MyModel(feature_num=input_size, d_model=64, nhead=2, num_layers=1)
    optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=1e-5)
    loss = nn.L1Loss().to(device)
    #loss = nn.L1Loss()

    out_path = "model/"
    if not os.path.exists(out_path):
        os.makedirs(out_path)
    best_mae = np.inf

    print(f'Train start...')
    for epoch in range(n_epochs):
        model.train()
        train_losses = []
        train_mae = []
        train_rmse = []
        batch_num = len(train_loader)

        # Training
        for X, y in train_loader:
            optimizer.zero_grad()
            outputs = model(X).squeeze()
            l = loss(outputs, y)
            l.backward()
            nn.utils.clip_grad_norm_(model.parameters(), max_norm=1)
            optimizer.step()
            train_losses.append(l.item())

            # Calculate MAE
            mae = mean_absolute_error(y.cpu().numpy(), outputs.detach().cpu().numpy())
            train_mae.append(mae)

            # Calculate RMSE
            rmse = math.sqrt(mean_squared_error(y.cpu().numpy(), outputs.detach().cpu().numpy()))
            train_rmse.append(rmse)

        epoch_train_loss = np.mean(train_losses)
        epoch_train_mae = np.mean(train_mae)
        epoch_train_rmse = np.mean(train_rmse)

        tot_train_losses.append(epoch_train_loss)
        tot_train_mae.append(epoch_train_mae)
        tot_train_rmse.append(epoch_train_rmse)

        print(f'Epoch [{epoch+1}/{n_epochs}] Training Loss: {epoch_train_loss:.4f}')
        print(f'Epoch [{epoch+1}/{n_epochs}] Training MAE: {epoch_train_mae:.4f}')
        print(f'Epoch [{epoch+1}/{n_epochs}] Training RMSE: {epoch_train_rmse:.4f}')

        train_maes = []

        model.eval()
        with torch.no_grad():
            valid_losses = []
            valid_maes = []
            valid_rmse = []
            valid_rmsle = []

            for X_v, y_v in valid_loader:
                preds = model(X_v).squeeze()
                valid_loss = loss(preds, y_v)
                valid_losses.append(valid_loss.item())

                # Calculate MAE
                valid_mae = mean_absolute_error(y_v.cpu().numpy(), preds.cpu().numpy())
                valid_maes.append(valid_mae)

                # Calculate RMSE
                valid_rmse_val = math.sqrt(mean_squared_error(y_v.cpu().numpy(), preds.cpu().numpy()))
                valid_rmse.append(valid_rmse_val)

            epoch_valid_loss = np.mean(valid_losses)
            epoch_valid_mae = np.mean(valid_maes)
            epoch_valid_rmse = np.mean(valid_rmse)

            tot_valid_losses.append(epoch_train_loss)
            tot_valid_mae.append(epoch_train_mae)
            tot_valid_rmse.append(epoch_train_rmse)

            print(f'Epoch [{epoch+1}/{n_epochs}] Validation Loss: {epoch_valid_loss:.4f}')
            print(f'Epoch [{epoch+1}/{n_epochs}] Validation MAE: {epoch_valid_mae:.4f}')
            print(f'Epoch [{epoch+1}/{n_epochs}] Validation RMSE: {epoch_valid_rmse:.4f}')


            if epoch_valid_mae < best_mae:
                best_mae = epoch_valid_mae
                torch.save(model, os.path.join(out_path, f"model_epoch_{epoch+1}.pt"))

        if epoch_valid_mae - pre_epoch_valid_mae > 0:
            patience_counter += 1

            if patience_counter == 2:
                lr = lr * 0.5
                patience_counter = 0
                for param_group in optimizer.param_groups:
                    param_group['lr'] = lr
                    print(f'renew lr to {lr}')

        pre_epoch_valid_mae = epoch_valid_mae

        if (epoch_valid_mae - epoch_train_mae > 0.03) or (lr <1e-7):
            print('Early stop now.')
            break

    print(f'Train over.')

Train start...
Epoch [1/50] Training Loss: 6.3510
Epoch [1/50] Training MAE: 6.3510
Epoch [1/50] Training RMSE: 9.3998
Epoch [1/50] Validation Loss: 6.2801
Epoch [1/50] Validation MAE: 6.2801
Epoch [1/50] Validation RMSE: 9.2793
Epoch [2/50] Training Loss: 6.2812
Epoch [2/50] Training MAE: 6.2812
Epoch [2/50] Training RMSE: 9.2843
Epoch [2/50] Validation Loss: 6.2655
Epoch [2/50] Validation MAE: 6.2655
Epoch [2/50] Validation RMSE: 9.2652
Epoch [3/50] Training Loss: 6.2736
Epoch [3/50] Training MAE: 6.2736
Epoch [3/50] Training RMSE: 9.2776
Epoch [3/50] Validation Loss: 6.2593
Epoch [3/50] Validation MAE: 6.2593
Epoch [3/50] Validation RMSE: 9.2621
Epoch [4/50] Training Loss: 6.2707
Epoch [4/50] Training MAE: 6.2707
Epoch [4/50] Training RMSE: 9.2747
Epoch [4/50] Validation Loss: 6.2550
Epoch [4/50] Validation MAE: 6.2550
Epoch [4/50] Validation RMSE: 9.2592
Epoch [5/50] Training Loss: 6.2661
Epoch [5/50] Training MAE: 6.2661
Epoch [5/50] Training RMSE: 9.2702
Epoch [5/50] Validation L

In [ ]:
torch.save(model.state_dict(), os.path.join(out_path, "space_time_transformer.pt"))

In [ ]:
import pickle

metrics = {
    "Train Loss": tot_train_losses,
    "Train MAE": tot_train_mae,
    "Train RMSE": tot_train_rmse,
    "Validation Loss": tot_valid_losses,
    "Validation MAE": tot_valid_mae,
    "Validation RMSE": tot_valid_rmse
}

with open('transformer_training_metrics.pkl', 'wb') as f:
    pickle.dump(metrics, f)

In [3]:
import pandas as pd

data = {
    "Training Loss": [6.3510, 6.2812, 6.2736, 6.2707, 6.2661, 6.2634, 6.2592, 6.2567, 6.2554, 6.2539,
                      6.2479, 6.2473, 6.2461, 6.2455, 6.2447, 6.2447, 6.2434, 6.2429, 6.2419, 6.2415,
                      6.2410, 6.2396, 6.2392, 6.2389, 6.2390, 6.2385, 6.2384, 6.2386, 6.2372, 6.2372,
                      6.2371, 6.2365, 6.2368, 6.2362, 6.2362, 6.2360, 6.2360, 6.2363, 6.2363, 6.2360,
                      6.2365, 6.2363, 6.2360, 6.2359, 6.2356, 6.2358, 6.2357, 6.2356, 6.2358, 6.2357],
    "Training MAE": [6.3510, 6.2812, 6.2736, 6.2707, 6.2661, 6.2634, 6.2592, 6.2567, 6.2554, 6.2539,
                     6.2479, 6.2473, 6.2461, 6.2455, 6.2447, 6.2447, 6.2434, 6.2429, 6.2419, 6.2415,
                     6.2410, 6.2396, 6.2392, 6.2389, 6.2390, 6.2385, 6.2384, 6.2386, 6.2372, 6.2372,
                     6.2371, 6.2365, 6.2368, 6.2362, 6.2362, 6.2360, 6.2360, 6.2363, 6.2363, 6.2360,
                     6.2365, 6.2363, 6.2360, 6.2359, 6.2356, 6.2358, 6.2357, 6.2356, 6.2358, 6.2357],
    "Training RMSE": [9.3998, 9.2843, 9.2776, 9.2747, 9.2702, 9.2675, 9.2633, 9.2607, 9.2589, 9.2570,
                      9.2512, 9.2498, 9.2482, 9.2475, 9.2465, 9.2465, 9.2451, 9.2446, 9.2429, 9.2425,
                      9.2420, 9.2409, 9.2400, 9.2389, 9.2398, 9.2385, 9.2385, 9.2386, 9.2386, 9.2369,
                      9.2370, 9.2368, 9.2368, 9.2362, 9.2362, 9.2357, 9.2357, 9.2351, 9.2352, 9.2348,
                      9.2352, 9.2352, 9.2348, 9.2349, 9.2344, 9.2346, 9.2343, 9.2343, 9.2345, 9.2342],
    "Validation Loss": [6.2801, 6.2655, 6.2593, 6.2550, 6.2511, 6.2468, 6.2474, 6.2456, 6.2456, 6.2442,
                        6.2378, 6.2361, 6.2333, 6.2338, 6.2345, 6.2345, 6.2332, 6.2331, 6.2314, 6.2324,
                        6.2330, 6.2308, 6.2284, 6.2281, 6.2267, 6.2277, 6.2265, 6.2275, 6.2279, 6.2265,
                        6.2265, 6.2265, 6.2258, 6.2254, 6.2250, 6.2253, 6.2250, 6.2253, 6.2251, 6.2252,
                        6.2253, 6.2250, 6.2253, 6.2250, 6.2252, 6.2250, 6.2250, 6.2253, 6.2252, 6.2253],
    "Validation MAE": [6.2801, 6.2655, 6.2593, 6.2550, 6.2511, 6.2468, 6.2474, 6.2456, 6.2456, 6.2442,
                       6.2378, 6.2361, 6.2333, 6.2338, 6.2345, 6.2345, 6.2332, 6.2331, 6.2314, 6.2324,
                       6.2330, 6.2308, 6.2284, 6.2281, 6.2267, 6.2277, 6.2265, 6.2275, 6.2279, 6.2265,
                       6.2265, 6.2265, 6.2258, 6.2254, 6.2250, 6.2253, 6.2250, 6.2253, 6.2251, 6.2252,
                       6.2253, 6.2250, 6.2253, 6.2250, 6.2252, 6.2250, 6.2250, 6.2253, 6.2252, 6.2253],
    "Validation RMSE": [9.2744, 9.2678, 9.2632, 9.2595, 9.2559, 9.2518, 9.2525, 9.2509, 9.2508, 9.2495,
                        9.2441, 9.2422, 9.2394, 9.2399, 9.2407, 9.2406, 9.2395, 9.2393, 9.2380, 9.2389,
                        9.2394, 9.2373, 9.2348, 9.2344, 9.2329, 9.2340, 9.2327, 9.2339, 9.2343, 9.2328,
                        9.2329, 9.2328, 9.2323, 9.2320, 9.2315, 9.2318, 9.2315, 9.2318, 9.2316, 9.2317,
                        9.2318, 9.2315, 9.2318, 9.2315, 9.2317, 9.2315, 9.2315, 9.2318, 9.2317, 9.2318]
}

In [2]:
metrics_df = pd.DataFrame(data)
metrics_df

,Epoch,Training Loss,Training MAE,Training RMSE,Validation Loss,Validation MAE,Validation RMSE
0,1,6.3510,6.3510,9.3998,6.2801,6.2801,9.2744
1,2,6.2812,6.2812,9.2843,6.2655,6.2655,9.2678
2,3,6.2736,6.2736,9.2776,6.2593,6.2593,9.2632
3,4,6.2707,6.2707,9.2747,6.2550,6.2550,9.2595
4,5,6.2661,6.2661,9.2702,6.2511,6.2511,9.2559
5,6,6.2634,6.2634,9.2675,6.2468,6.2468,9.2518
6,7,6.2592,6.2592,9.2633,6.2474,6.2474,9.2525
7,8,6.2567,6.2567,9.2607,6.2456,6.2456,9.2509
8,9,6.2554,6.2554,9.2589,6.2456,6.2456,9.2508
9,10,6.2539,6.2539,9.2570,6.2442,6.2442,9.2495


In [4]:
metrics_df.min()

Epoch              1.0000
Training Loss      6.2356
Training MAE       6.2356
Training RMSE      9.2342
Validation Loss    6.2250
Validation MAE     6.2250
Validation RMSE    9.2315
dtype: float64

In [ ]:
metrics_df = pd.DataFrame(metrics)
metrics_df.to_csv('transformer_training_metrics.csv', index=False)

In [ ]:
metrics_df = pd.read_csv('transformer_training_metrics.csv')
metrics_df

,Train Loss,Train MAE,Train RMSE,Validation Loss,Validation MAE,Validation RMSE
0,6.350999,6.350999,9.399841,6.350999,6.350999,9.399841
1,6.281198,6.281198,9.284310,6.281198,6.281198,9.284310
2,6.273588,6.273588,9.277553,6.273588,6.273588,9.277553
3,6.270671,6.270671,9.274746,6.270671,6.270671,9.274746
4,6.266131,6.266130,9.270213,6.266131,6.266130,9.270213
5,6.263353,6.263353,9.267455,6.263353,6.263353,9.267455
6,6.259235,6.259235,9.263309,6.259235,6.259235,9.263309
7,6.256734,6.256733,9.260707,6.256734,6.256733,9.260707
8,6.255377,6.255377,9.258885,6.255377,6.255377,9.258885
9,6.253917,6.253917,9.257002,6.253917,6.253917,9.257002


In [ ]:
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

class TimeSeriesTransformer(nn.Module):
    def __init__(self, num_features, num_layers=3, d_model=64, nhead=4, dim_feedforward=256, dropout=0.1):
        super(TimeSeriesTransformer, self).__init__()
        self.linear_in = nn.Linear(num_features, d_model)
        self.transformer_encoder_layer = nn.TransformerEncoderLayer(
            d_model=d_model, nhead=nhead,
            dim_feedforward=dim_feedforward, dropout=dropout)
        self.transformer_encoder = nn.TransformerEncoder(
            self.transformer_encoder_layer, num_layers=num_layers)
        self.linear_out = nn.Linear(d_model, 1)

    def forward(self, src):
        src = self.linear_in(src)  # [batch_size, seq_len, d_model]
        src = src.permute(1, 0, 2)  # [seq_len, batch_size, d_model]
        output = self.transformer_encoder(src)
        output = output.permute(1, 0, 2)  # [batch_size, seq_len, d_model]
        output = self.linear_out(output[:, -1, :])  # Use the output of the last time step
        return output


# Initialize the model
model = TimeSeriesTransformer(num_features=X_train.shape[1]).to(device)

# Define Loss Function and Optimizer
loss_function = torch.nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# DataLoader
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)


def check_nan_inf(tensor):
    if torch.isnan(tensor).any() or torch.isinf(tensor).any():
        return True
    return False

# Check for NaN or Inf in data
assert not check_nan_inf(X_train), "NaN or Inf in X_train"
assert not check_nan_inf(y_train), "NaN or Inf in y_train"
assert not check_nan_inf(X_val), "NaN or Inf in X_val"
assert not check_nan_inf(y_val), "NaN or Inf in y_val"

# Training loop with gradient clipping
for epoch in range(10):
    model.train()
    for features, targets in train_loader:
        features, targets = features.to(device), targets.to(device)
        optimizer.zero_grad()
        outputs = model(features)

        if torch.isnan(outputs).any():
              print("NaN detected in model outputs")
              print("Features causing NaN:", features)
              raise ValueError("NaN detected in model outputs")

        loss = loss_function(outputs, targets)

        if torch.isnan(loss).any():
            raise ValueError("NaN detected in loss")

        loss.backward()

        # Gradient Clipping
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)

        optimizer.step()

    print(f"Epoch {epoch+1}, Loss: {loss.item()}")

    # Validation
    model.eval()
    val_loss = 0
    with torch.no_grad():
        for features, targets in val_loader:
            features, targets = features.to(device), targets.to(device)
            outputs = model(features)
            val_loss += loss_function(outputs, targets).item()

            if torch.isnan(outputs).any():
              print("NaN detected in model outputs")
              print("Features causing NaN:", features)
              raise ValueError("NaN detected in model outputs")

    val_loss /= len(val_loader)
    print(f"Validation Loss: {val_loss}")

NaN detected in model outputs
Features causing NaN: tensor([[[1.1200e+02, 3.9000e+02, 5.3307e+06,  ..., 2.0001e+00,
          4.9149e-03, 2.0947e-02]],

        [[1.4200e+02, 5.1000e+02, 1.4141e+06,  ..., 2.0001e+00,
          6.4599e-03, 5.5993e-02]],

        [[5.4000e+01, 4.2000e+02, 3.9293e+06,  ..., 1.9999e+00,
          5.1270e-03, 1.9226e-02]],

        ...,

        [[7.2000e+01, 4.9000e+02, 4.3639e+05,  ..., 1.9998e+00,
          5.4252e-03, 1.6973e-02]],

        [[1.1600e+02, 4.1000e+02, 5.9743e+06,  ..., 1.9995e+00,
          4.6003e-03, 1.9872e-02]],

        [[1.1000e+02, 4.5000e+02, 1.4784e+07,  ..., 1.9991e+00,
          4.1548e-03, 1.9975e-02]]], device='cuda:0')


ValueError: ignored